# Self-Organizing Maps (SOMs) - Practical Part

## Example 1 - Iris dataset

Here is an example of using SOMs on the Iris dataset to classify different species based on their petal and sepal measurements.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler

import math
import numpy as np

import matplotlib.pyplot as plt

## Preparing the data

In [ ]:
iris = load_iris()
features = iris.data
labels = iris.target
labels

## Normalizing the data

In [ ]:
sc = MinMaxScaler()
features = sc.fit_transform(features)

## Building the SOM

### Train the SOM

The Minisom package allows us to create SOMs easily:

-   (x, y) represent the dimensions of the map.
-   input\_len is the number of features in the data

> A recommended map size is: 5 \* sqrt(number of samples). If you have 150 samples, the map size is 64 or 8x8.

In [ ]:
# !pip install minisom
from minisom import MiniSom

In [ ]:
n_features = features.shape[1]
n_samples = features.shape[0]

In these lines:

-   **`n_features = features.shape[1]`**: This retrieves the number of features in the dataset by accessing the second dimension of the `features` array (assuming `features` is a two-dimensional array where rows represent samples and columns represent features).
-   **`n_samples = features.shape[0]`**: This retrieves the number of samples (or data points) in the dataset by accessing the first dimension of the `features` array.

In [ ]:
map_size = 5 * math.sqrt(n_samples)
map_height = map_width = math.ceil(math.sqrt(map_size))

-   **`map_size = 5 * math.sqrt(n_samples)`**: This line calculates an initial size for the SOM. It scales the size of the map relative to the square root of the number of samples, multiplied by 5. This heuristic helps to determine an appropriate size of the map that can effectively model the data complexity and density.
-   **`map_height = map_width = math.ceil(math.sqrt(map_size))`**: This line sets both the height and width of the SOM to be the ceiling value of the square root of `map_size`. The use of `math.ceil` ensures that the map dimensions are whole numbers. Making the map height and width equal suggests a square grid for the SOM, which is common in many applications.

In [ ]:
print(f'(map_height, map_width) = ({map_height}, {map_width})')
print(f'Number of features: {n_features}')

In [ ]:
som = MiniSom(x=map_height, y=map_width, input_len=n_features, sigma=1.5, learning_rate=0.5, neighborhood_function='gaussian', random_seed=1)

In this line, an instance of the **Self-Organizing Map** (SOM) is created using the `MiniSom` library. The parameters are defined as follows:

-   **`x=map_height`**: Sets the height of the SOM grid. This defines the number of neurons along one dimension of the map.
-   **`y=map_width`**: Sets the width of the SOM grid. This defines the number of neurons along the other dimension of the map.
-   **`input_len=n_features`**: Specifies the number of features in the input dataset. Each neuron's weight vector will have this dimensionality.
-   **`sigma=1.5`**: Determines the radius of the neighborhood function in the SOM. It influences how many neurons around the Best Matching Unit (BMU) will be updated during each training iteration.
-   **`learning_rate=0.5`**: Controls the learning rate of the SOM. It affects how much the neuron weights are adjusted during each training step.
-   **`neighborhood_function='gaussian'`**: Indicates the type of neighborhood function to use when updating neuron weights. 'Gaussian' is a common choice that weights the influence of the BMU on neighboring neurons based on their distance from the BMU.
-   **`random_seed=1`**: Sets the seed for random number generation, ensuring reproducibility of the results.

In [ ]:
som.pca_weights_init(features)

This line initializes the weights of the SOM using __Principal Component Analysis (PCA)__. This method sets the initial weights to mirror the principal components of the data, which can lead to faster and more stable convergence in some cases.

In [ ]:
som.train(data=features, num_iteration=1000, verbose=True)

-   **`data=features`**: The dataset used to train the SOM. Each element of the dataset should be a vector of length `n_features`.
-   **`num_iteration=1000`**: The total number of iterations for which the SOM will be trained.
-   **`verbose=True`**: When set to `True`, the training process will print out log messages about the progress, which is helpful for tracking the training process.
-   **`quantization error`**: This is a measure of the average distance between each data point and its __Best Matching Unit (BMU)__ in the SOM. The quantization error is a key metric used to evaluate the performance of the SOM. It reflects __how well the SOM has learned to represent the dataset__.

## Visualzing the results
We can cleary dsitinguish the clusters:
* __Setosa__: red circles
* __Versicolor__:  green squares
* __Virginica__: blue triangles 

In [ ]:
print(iris.target_names)

In [ ]:
plt.figure(figsize=(6, 4))

# plot U-matrix
u_matrix = som.distance_map().T
plt.pcolor(u_matrix, cmap='bone_r')
plt.colorbar()

# plot markers
markers = ['o', 's', '^']   # 'setosa', 'versicolor' 'virginica'
colors = ['r', 'g', 'b']
for feature, label in zip(features, labels):
    w = som.winner(feature)
    plt.plot(w[0] + 0.5, w[1] + 0.5, 
        markers[label], markeredgecolor = colors[label], 
        markerfacecolor = 'None', markersize = 10, markeredgewidth = 1)

plt.show()

## Plotting the clusters
On our __distance map__, the __light shades__ represent the __cluster centrois__ whereas the __dark ones__ represents the __division of the clusters__. 

However, some people plot their distance map the other way where dark shades represents clusters.

In [ ]:
def plot_distance_map(ax, fig):
    """Plot the distance map"""
    p = ax.pcolor(som.distance_map().T, cmap='bone_r') # cmap='Blues'
    # ax.colorbar()
    fig.colorbar(p, ax=ax)

In [ ]:
def plot_clusters_scatter(ax):
    """
    Create a scatter plot of the winning neurons. 
    Each neuron is assigned the color of the cluster it belongs to.
    """ 
    # Get the winning neuron coordinates for each sample 
    # The coordinates are transformed into an array for the scatter plot: (1,1) => [1,1]
    winning_neurons = np.array([som.winner(x) for x in features])

    # Add a random offset to avoid overlaps between points within the same cell
    offset = np.random.uniform(low=-0.4, high=0.4, size=(len(features), 2))
    winning_neurons = winning_neurons + offset

    # Define the colors based on the labels
    colors = ['#ff0400', 'g', '#e88325']
    label_colors = [colors[label] for label in labels]

    # Create the scatter plot
    # 1st column represent x and second, y coordinate
    ax.scatter(winning_neurons[:,0], winning_neurons[:,1], s=10, c=label_colors)

In [ ]:
def plot_clusters_markers(ax):
    """
    Plot the winning neurons as markers.
    Each marker is assigned the color of the cluster ir belongs to.
    """
    markers = ['o', 's', '^']
    colors = ['#ff0400', 'g', '#e88325']
    for i, feature in enumerate(features):
        w = som.winner(feature)
        ax.plot(w[0] + 0.5, w[1] + 0.5, 
            markers[labels[i]], markeredgecolor = colors[labels[i]], 
            markerfacecolor = 'None', markersize = 10, markeredgewidth = 1)
        
    # legend
    ax.legend(handles=[plt.Line2D([], [], color='#ff0400', marker='o', linestyle='None', label='Setosa'),
                    plt.Line2D([], [], color='green', marker='s', linestyle='None', label='Versicolor'),
                    plt.Line2D([], [], color='#e88325', marker='^', linestyle='None', label='Virginica')],
                    bbox_to_anchor=(1.5, 1.03))

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

plot_distance_map(axes[0], fig)
plot_clusters_scatter(axes[1])
plot_clusters_markers(axes[2])

plt.suptitle("Plants species clusters")
plt.show()

## Example 2 - Color clustering

Let's create a simple example where we use a SOM to cluster colors. We will generate some random colors, train a SOM, and then visualize how the SOM organizes these colors.

#### Import Libraries

We start by importing the necessary libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from minisom import MiniSom

#### Generate Random Colors

We will generate a dataset of random colors. Each color is represented as a three-dimensional vector (RGB) where each component is between 0 and 1.



In [ ]:
# Generate random colors
colors = np.random.rand(100, 3)  # 100 random colors

#### Initialize and Train the SOM

We will create a SOM with a grid size that is appropriate for visualizing in a 2D space. Here, we'll use a $8*8$ grid -> $5 * \sqrt(n\_samples)$.

In [ ]:
map_size = 5 * math.sqrt(len(colors))
map_height = map_width = math.ceil(math.sqrt(map_size))
print(f'(map_height, map_width) = ({map_height}, {map_width})')
print(f'Number of features: {colors.shape[1]}')

In [ ]:
# SOM initialization and training

som_cc = MiniSom(map_width, map_height, 3, sigma=1.5, learning_rate=0.5, neighborhood_function='gaussian', random_seed=1)  # 8x8 grid, input size=3 (RGB)
som_cc.random_weights_init(colors)

n_interations_cc = 1000 # train with 1000 iterations
quantization_errors_cc = []
for i in range(n_interations_cc):
    som_cc.train_random(colors, i+1)  # Train with 1 iteration at a time
    quantization_errors_cc.append(som_cc.quantization_error(colors))

### Quantization error over time

In [ ]:
# Plot quantization error over iterations
plt.figure(figsize=(10, 5))
plt.plot(quantization_errors_cc)
plt.title('Quantization Error Over Iterations')
plt.xlabel('Iteration')
plt.ylabel('Quantization Error')
plt.grid(True)
plt.show()

#### Visualize the Results

Finally, we'll visualize the results. We'll plot the SOM's grid, where each node's color corresponds to the neuron's weight (which represents a color).

This visualization shows how the SOM has organized similar colors together, creating a sort of "map" of colors. It's a powerful way to see the capability of SOMs in organizing unlabelled data in a meaningful way.

In [ ]:
# Plotting the colors
plt.figure(figsize=(7, 7))
plt.title('Color clustering using SOM')
for x in range(map_width):
    for y in range(map_height):
        w = som_cc.get_weights()[x, y]
        plt.plot(x+0.5, y+0.5, 'o', markerfacecolor=tuple(w), markeredgecolor='k', markersize=10)

# Decoration settings
plt.xlim([0, map_width])
plt.ylim([0, map_height])
plt.show()

## Example 3 - Movie Recommendation System (Synthetic)

#### Generate Movie Data

In [ ]:
from sklearn.datasets import make_blobs

# Generate three clusters of movie data representing three genres
num_movies = 50

movie_titles = [
    "Space Adventure", "The Comedian", "Drama Queen", "Lonesome Hero", "Action Packed",
    "Laugh Out Loud", "Tears and Smiles", "Fast and Curious", "Mystery Shack", "Last Stand",
    "Silent Guardian", "Funny Bones", "Crying Out Loud", "Explosions Galore", "Rolling in Laughs",
    "Serious Business", "Joy Ride", "Thrill Seeker", "Quiet Desperation", "Blaze of Glory",
    "Comic Stand", "Dramatic Entrance", "Sudden Impact", "Laughing Stock", "Hero’s Journey",
    "Roaring Laughter", "Cry Me a River", "Punch Drunk", "Mystery Machine", "Standoff",
    "Hidden Agenda", "Serious Laughter", "River of Tears", "Thunderous Applause", "Giggle Fest",
    "Tragic Turn", "Joyful Noise", "Impact Zone", "Moment of Silence", "Epic Saga",
    "Belly Laughs", "Drama Club", "Action Hero", "Comedy Night", "Dramatic Pause",
    "Race Against Time", "Comic Relief", "Plot Twist", "Last Laugh", "Final Act"
]

genres = ['Action', 'Comedy', 'Drama']
centers = [[1, 1], [6, 1], [3, 6]]  # These can be thought of as genre centroids
cluster_std = [0.8, 0.5, 1]

movies, genre_indices = make_blobs(n_samples=num_movies, centers=centers, cluster_std=cluster_std, random_state=1)

In [ ]:
# Assuming genres corresponds to the indices for labels
movie_genres = np.array([genres[i] for i in genre_indices])

# Simulate user ratings (assuming 5 users)
num_users = 5
ratings = np.zeros((num_users, num_movies))

# Users have a preferred genre, more likely to rate those movies higher
user_pref_genres = np.random.choice(genres, num_users)

for user_idx in range(num_users):
    for movie_idx in range(num_movies):
        # Bias the ratings towards user's preferred genre
        bias = 1.0 if movie_genres[movie_idx] == user_pref_genres[user_idx] else 0.3
        ratings[user_idx, movie_idx] = np.random.rand() * bias

# Normalize ratings for combination with movie features
normalized_ratings = np.mean(ratings, axis=0).reshape(-1, 1) / np.max(np.mean(ratings, axis=0))

# Combine movie features and normalized ratings into a single feature array
features = np.hstack((movies, normalized_ratings))

#### Initialize and Train the SOM

In [ ]:
map_size = 5 * math.sqrt(num_movies)
map_height = map_width = math.ceil(math.sqrt(map_size))
print(f'(map_height, map_width) = ({map_height}, {map_width})')
print(f'Number of features: {features[0].shape[0]}')

In [ ]:
# Initialize and train the SOM
som_mr = MiniSom(map_width, map_height, 3, sigma=0.5, learning_rate=0.5, random_seed=1)
som_mr.random_weights_init(features)

n_interations_mr = 1000 # train with 1000 iterations
quantization_errors_mr = []
for i in range(n_interations_mr):
    som_mr.train_random(features, i+1)  # Train with 1 iteration at a time
    quantization_errors_mr.append(som_mr.quantization_error(features))

In [ ]:
# Plot quantization error over iterations
plt.figure(figsize=(10, 5))
plt.plot(quantization_errors_mr)
plt.title('Quantization Error Over Iterations')
plt.xlabel('Iteration')
plt.ylabel('Quantization Error')
plt.grid(True)
plt.show()

#### Visualize the Movie Clusters

In [ ]:
from matplotlib.lines import Line2D

# Plotting
plt.figure(figsize=(8, 6))
plt.title('Movie Clusters based on Genres and Ratings')

# Define colors for each genre
colors = ['red', 'green', 'blue']
genre_labels = ['Action', 'Comedy', 'Drama']  # Labels corresponding to the genres

# Map genre indices to color
color_map = {genres.index(genre): colors[i] for i, genre in enumerate(genre_labels)}

# Plot all movies on the map according to their BMU (Best Matching Unit)
for cnt, feature in enumerate(features):
    bmu = som_mr.winner(feature)  # Find the best matching unit for each movie
    genre_index = genre_indices[cnt]  # Use genre_indices from the blob generation
    color = color_map[genre_index]  # Use the color map
    plt.plot(bmu[0] + 0.5, bmu[1] + 0.5, 'o', markerfacecolor=color, markeredgecolor='k', markersize=18)

# Mark nodes in the SOM
for x in range(som_mr.get_weights().shape[0]):
    for y in range(som_mr.get_weights().shape[1]):
        plt.text(x + 0.5, y + 0.5, 'x', color='k', ha='center', va='center', fontweight='bold')

# Adding custom legend entries
legend_elements = [Line2D([0], [0], marker='o', color='w', label=genre, markerfacecolor=color_map[genres.index(genre)], markersize=10) for genre in genre_labels]
plt.legend(handles=legend_elements, bbox_to_anchor=(1, 1.02))

plt.xlim([0, som_mr.get_weights().shape[0]])
plt.ylim([0, som_mr.get_weights().shape[1]])
plt.xticks([])
plt.yticks([])
plt.grid(False)
plt.show()

#### Recommender Function

Define a function to recommend movies based on user preferences.

In [ ]:
def recommend_movies(user_ratings, som, features, movie_titles, threshold=0.5):
    # Calculate user preference vector by averaging movies they rate highly
    preferred_movies = user_ratings > threshold  # Movies above threshold
    if np.any(preferred_movies):
        user_profile = np.average(features[preferred_movies, :], axis=0, weights=user_ratings[preferred_movies])
    else:
        user_profile = np.average(features, axis=0)  # Fallback to overall average if no preferred movies

    # Find the best matching unit for the user profile
    winner = som.winner(user_profile)

    # Find titles of movies in the same node
    recommended_titles = [movie_titles[i] for i in range(len(features)) if som.winner(features[i]) == winner]

    return recommended_titles

# Example usage
user_ratings = ratings[0]  # Assume we're recommending for the first user
recommended_titles = recommend_movies(user_ratings, som_mr, features, movie_titles)
print("Recommended Movie Titles:", recommended_titles)